In [54]:
import numpy as np
import pandas as pd

from sklearn.utils.validation import check_is_fitted, check_array , check_X_y
from sklearn.base import clone
from sklearn.exceptions import NotFittedError

In [ ]:
'''
1.Initialization
- initialize weights for all instances to be equal : 1/m
- empty lists to store alphas and classifiers

2.fit() method
- train weak classfier using current weights(alphas)
- calculate weighted errors 
- calculate alphas based on error rate
- update weights of the instances for the next iteration
- normalize weights
- store trained classifiers and alphas

3. predict() method
- aggregate predictions of all classifiers using their alphas
- return final prediction as sign of the sum of weighted predictions

4. predict_proba() method
- return probability of the final prediction as sum of weighted probabilities
'''

'\n1.Initialization\n- initialize weights for all instances to be equal : 1/m\n- empty lists to store alphas and classifiers\n\n2.fit() method\n- train weak classfier using current weights(alphas)\n- calculate weighted errors \n- calculate alphas based on error rate\n- update weights of the instances for the next iteration\n- normalize weights\n- store trained classifiers and alphas\n\n3. predict() method\n- aggregate predictions of all classifiers using their alphas\n- return final prediction as sign of the sum of weighted predictions\n'

In [94]:
class adab():
    def __init__(self, weak_classifier, n_classifiers,learning_rate=1.0):
        self.weak_classifier = weak_classifier #weak classifier to be used
        self.n_classifiers = n_classifiers
        self.alphas = []
        self.classifiers = []
        self.learning_rate = learning_rate
        self.proba = None
        self.is_fitted_ = False
    def fit(self,X,y):

        #first check if X and y have the same length
        X,y = check_X_y(X,y)

        #initialize weights
        m,n = X.shape
        w = np.ones(m) / m
        self.alphas = []
        self.classifiers = []

        for t in range(self.n_classifiers):
            clf = self.weak_classifier()
            clf.fit(X,y,sample_weight=w)
            y_pred = clf.predict(X)

            # 1.weighted error rates
            # error = sum of weights of misclassified instances
            r = np.sum(w*(y_pred != y))

            # 2. predictor weights (alpha)
            # alpha = learning_rate * ln((1-error)/error)
            # to avoid division by zero, add a small value to error
            alpha = self.learning_rate * np.log((1-r)/(r+1e-10))
            #store alpha
            self.alphas.append(alpha)
            #store classifier
            self.classifiers.append(clf)

            # 3. update weights
            # increase weights of misclassified instances
            # decrease weights of correctly classified instances
            w = w * np.exp(alpha * (y_pred != y))
            # normalize weights
            w = w / np.sum(w)

        self.is_fitted_ = True

        # fit method always returns self
        return self

    def predict(self,X):
        # check if fitted 
        check_is_fitted(self,'is_fitted_')

        # validate X input
        X = check_array(X)

        #get predictions from all stored weak_classifiers
        clf_preds = np.array([alpha * clf.predict(X) for alpha,clf in zip(self.alphas, self.classifiers)])
        #return sign of sum of weighted predictions
        y_pred = np.sign(np.sum(clf_preds,axis=0))

        return y_pred
    
    def predict_proba(self,X):
        # check if fitted 
        check_is_fitted(self,'is_fitted_')

        # validate X input
        X = check_array(X)

        #get predictions from all stored weak_classifiers
        clf_preds = np.array([alpha * clf.predict(X) for alpha,clf in zip(self.alphas, self.classifiers)])
        #return sign of sum of weighted predictions
        proba = np.sum(clf_preds,axis=0)

        proba = (proba - proba.min())/proba.ptp()
        return np.vstack([1-proba,proba]).T


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [58]:
X,y = make_classification(n_samples=1000,n_features=20,n_informative=10,n_classes=2,random_state=42)

X.shape , y.shape

((1000, 20), (1000,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(800, 20) (200, 20) (800,) (200,)


In [116]:
clf1 = adab(weak_classifier= lambda: DecisionTreeClassifier(random_state=42,
                                                            max_depth=20,
                                                            splitter='random',
                                                            criterion='entropy'),

            n_classifiers=1000,
            learning_rate=0.5)

clf1.fit(X_train,y_train)

y_pred = clf1.predict(X_test)


In [108]:
accuracy_score(y_test,y_pred)

0.755

In [115]:
clf1.classifiers[50].get_depth()

16

In [80]:
y_proba = clf1.predict_proba(X_test)
y_proba[:15]

array([[0.   , 1.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.   , 1.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.323, 0.677],
       [0.   , 1.   ],
       [0.   , 1.   ],
       [1.   , 0.   ],
       [0.   , 1.   ],
       [0.   , 1.   ]])

In [99]:
clf1.alphas[:10]

[11.512925464970229,
 11.512925464970229,
 11.512925464970229,
 11.512925464970229,
 11.512925464970229,
 11.512925464970229,
 11.512925464970229,
 11.512925464970229,
 11.512925464970229,
 11.512925464970229]

In [100]:
from sklearn.ensemble import AdaBoostClassifier

In [101]:
clf2 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=42),
                            n_estimators=1000,
                            learning_rate=0.5)

clf2.fit(X_train,y_train)

y_pred2 = clf2.predict(X_test)

accuracy_score(y_test,y_pred2)

c:\Users\aarad\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.845